###1. Transform customers data


#####1.1 Remove records with null customer_id values

In [0]:
SELECT * FROM gizmobox1.bronze.v_customers WHERE customer_id IS NOT NULL;

#####1.2 Remove exact duplicates

In [0]:
CREATE TEMPORARY VIEW v_customers_distinct
AS
SELECT DISTINCT * 
FROM gizmobox1.bronze.v_customers 
WHERE customer_id IS NOT NULL;

#####1.3 Remove duplicates based on created_timestamp, cast columns to correct types

In [0]:
WITH CTE_MAX AS 
  (SELECT customer_id,
            MAX(created_timestamp) as created_timestamp
   FROM gizmobox1.bronze.v_customers 
   GROUP BY customer_id)
SELECT
  CAST(t.created_timestamp AS TIMESTAMP) AS created_timestamp,
  t.customer_id,
  t.customer_name,
  CAST(t.date_of_birth AS DATE) AS date_of_birth,
  t.email,
  CAST(t.member_since AS DATE) AS member_since,
  t.telephone
FROM
  v_customers_distinct t
JOIN
  CTE_MAX m
  ON t.customer_id = m.customer_id
  AND t.created_timestamp = m.created_timestamp;


###2. Write data to table in silver layer

In [0]:
CREATE TABLE gizmobox1.silver.customers
AS
WITH CTE_MAX AS 
  (SELECT customer_id,
            MAX(created_timestamp) as created_timestamp
   FROM gizmobox1.bronze.v_customers 
   GROUP BY customer_id)
SELECT
  CAST(t.created_timestamp AS TIMESTAMP) AS created_timestamp,
  t.customer_id,
  t.customer_name,
  CAST(t.date_of_birth AS DATE) AS date_of_birth,
  t.email,
  CAST(t.member_since AS DATE) AS member_since,
  t.telephone
FROM
  v_customers_distinct t
JOIN
  CTE_MAX m
  ON t.customer_id = m.customer_id
  AND t.created_timestamp = m.created_timestamp;


#####2.1 Check data in created table

In [0]:
SELECT * FROM gizmobox1.silver.customers;